# <center> Scraping and Merging NFL Data (For 2025 Season)</center>

In [1]:
# Import Python Libraries

import numpy as np
import pandas as pd
import random
import time

In [2]:
# Create list of teams for Pro-Football-Reference Datapull

teams = [
        'crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle',
        'dal', 'den', 'det', 'gnb', 'htx', 'clt', 'jax', 'kan',
        'sdg', 'ram', 'rai', 'mia', 'min', 'nwe', 'nor', 'nyg',
        'nyj', 'phi', 'pit', 'sfo', 'sea', 'tam', 'oti', 'was'
]

print(f'number of teams = {len(teams)}')

number of teams = 32


In [3]:
rename_dict = {
     'Unnamed: 5':'Home', 'Rslt':'Win', 'Pts':'Tm_Pts', 'Pts0':'Opp_Pts',
     'Cmp':'pCmp', 'Att':'pAtt', 'Cmp%':'pCmp%', 'Yds':'pYds',
     'TD':'pTD', 'Y/A':'pY/A', 'AY/A':'pAY/A', 'Rate':'pRate', 
     'Yds.1':'SkYds', 'Att.1':'rAtt', 'Yds.2':'rYds', 'TD.1':'rTD', 
     'Y/A.1':'rY/A', 'Yds.3':'PntYds', 'Pass':'fdPass', 'Rsh':'fdRush',
     'Pen':'fdPen', 'Pen.1':'Pen', 'Yds.4':'PenYds'
}

## 1. Scrape NFL Gamelog Data

In [ ]:
# Get the starting time
start_time = time.time()

# Create the list of seasons to download - change to 2026 when wanting to grab 2025 season
seasons = range(2015, 2025)

# Create an empty dataframe to append
nfl_df = pd.DataFrame()

# Iterate through the seasons
for season in seasons:
    # Iterate through the teams
    for team in teams:
        # Set URL
        url = 'https://www.pro-football-reference.com/teams/' + team + '/' + str(season) + '/gamelog/'
        print(url)

        # Get Team Gamelog Data

        # Get Team Stats (from Game Logs table)
        table_id = 'table_pfr_team-year_game-logs_team-year-regular-season-game-log'
        tm_df = pd.read_html(url, header=1, attrs={'id': table_id})[0]

        # Drop rows where the 'Rk" value is NaN and rename some columns
        tm_df = tm_df.dropna(subset=['Rk'])
        tm_df = tm_df.rename(rename_dict, axis=1)

        # Add 'Tm_' prefix to game stats
        pre_dict = {col:f'Tm_{col}' for col in tm_df.columns[11:]}
        tm_df = tm_df.rename(columns=pre_dict)

        # Get Opponent Gamelog Data
        
        # Get Opponent Stats (from Opponent Game Logs Table)
        table_id = 'table_pfr_team-year_game-logs_team-year-regular-season-opponent-game-log'
        opp_df = pd.read_html(url, header=1, attrs={'id': table_id})[0]

        # Drop rows where the 'Rk" value is NaN and rename some columns
        opp_df = opp_df.dropna(subset=['Rk'])
        opp_df = opp_df.rename(rename_dict, axis=1)

        # Add 'Tm_' prefix to game stats
        pre_dict = {col:f'Opp_{col}' for col in opp_df.columns[11:]}
        opp_df = opp_df.rename(columns=pre_dict)

        # Merge the Two Dataframes (tm_df and opp_df)

        # Create Columns to merge
        cols_to_merge = tm_df.columns[:11].tolist()

        # Merge based on the first eleven colunms
        merged_df = pd.merge(tm_df, opp_df, on=cols_to_merge)

        # Insert Season and Team as new columns
        merged_df.insert(loc=0, column='Season', value=season)
        merged_df.insert(loc=1, column='Team', value=team.upper())

        # Append to NFL_DF (aggregate dataframe)

        # Concatenate the team gamelog to the aggregate dataframe
        nfl_df = pd.concat([nfl_df, merged_df], ignore_index=True)

        # Pause the program to abide by the website's rules on (FBref and Stathead sites no more than 10 requests in a minute)
        time.sleep(random.randint(4, 5))

# Get the ending time
end_time = time.time()

# Display the elapsed time
print(f'Elapsed time: {end_time - start_time:1f} seconds')

print(nfl_df.info())

https://www.pro-football-reference.com/teams/crd/2015/gamelog/
https://www.pro-football-reference.com/teams/atl/2015/gamelog/
https://www.pro-football-reference.com/teams/rav/2015/gamelog/
https://www.pro-football-reference.com/teams/buf/2015/gamelog/
https://www.pro-football-reference.com/teams/car/2015/gamelog/
https://www.pro-football-reference.com/teams/chi/2015/gamelog/
https://www.pro-football-reference.com/teams/cin/2015/gamelog/
https://www.pro-football-reference.com/teams/cle/2015/gamelog/
https://www.pro-football-reference.com/teams/dal/2015/gamelog/
https://www.pro-football-reference.com/teams/den/2015/gamelog/
https://www.pro-football-reference.com/teams/det/2015/gamelog/
https://www.pro-football-reference.com/teams/gnb/2015/gamelog/
https://www.pro-football-reference.com/teams/htx/2015/gamelog/
https://www.pro-football-reference.com/teams/clt/2015/gamelog/
https://www.pro-football-reference.com/teams/jax/2015/gamelog/
https://www.pro-football-reference.com/teams/kan/2015/g

In [7]:
# Clean Data

# Drop the Rk column
nfl_df = nfl_df.drop(columns=['Rk'], axis=1)

# Convert 'Home' column to 1 = Home or 0 = Away
nfl_df['Home'] = np.where(nfl_df['Home'] == '@', 0, 1)

# Convert 'Win' column to 1 = Win or 0 = Loss (or Tie)
nfl_df['Win'] = np.where(nfl_df['Win'] == 'W', 1, 0)

# Convert 'OT' column to 1 = OT or 0 = No OT
nfl_df['OT'] = np.where(nfl_df['OT'] == 'OT', 1, 0)

# Display updated info
print(nfl_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5246 entries, 0 to 5245
Data columns (total 86 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Season      5246 non-null   int64  
 1   Team        5246 non-null   object 
 2   Gtm         5246 non-null   float64
 3   Week        5246 non-null   float64
 4   Date        5246 non-null   object 
 5   Day         5246 non-null   object 
 6   Home        5246 non-null   int64  
 7   Opp         5246 non-null   object 
 8   Win         5246 non-null   int64  
 9   Tm_Pts      5246 non-null   int64  
 10  PtsO        5246 non-null   int64  
 11  OT          5246 non-null   int64  
 12  Tm_pCmp     5246 non-null   int64  
 13  Tm_pAtt     5246 non-null   int64  
 14  Tm_pCmp%    5246 non-null   float64
 15  Tm_pYds     5246 non-null   int64  
 16  Tm_pTD      5246 non-null   int64  
 17  Tm_pY/A     5246 non-null   float64
 18  Tm_pAY/A    5246 non-null   float64
 19  Tm_pRate    5246 non-null  

In [8]:
# Save the downloaded data to a CSV file
nfl_df.to_csv('nfl_gamelogs_2015-2024_NEW.csv', index=False)